## LC0

#### Import necessary libraries

In [ ]:
import chess
import chess.svg
import subprocess
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, SVG
import random
import matplotlib.pyplot as plt
import time
import numpy as np

### Initialization of Chess Board and Widgets

In [ ]:
board = chess.Board()
board_output = widgets.Output()
info_output = widgets.Output()

#### Display the last move made

In [ ]:
opponent_move_display = widgets.Text(
    description='Last Move:',
    disabled=True,
    layout=widgets.Layout(width='300px')
)

#### Input field for player's move

In [ ]:
move_input = widgets.Text(
    description='Move:',
    placeholder='e.g., e2e4',
    layout=widgets.Layout(width='200px')
)

### Function Definitions

#### Function to Display the Chessboard

In [ ]:
def show_board(board):
    svg_content = chess.svg.board(board=board, size=400)
    display(SVG(svg_content))

#### Function to Get the Best Move from the lc0 (one best move)

In [ ]:
def get_best_move(fen=None, time_limit=2000):
    global process
    lc0_path = Path("D:/GitHub/lc0/lc0.exe")
    if not lc0_path.exists():
        raise FileNotFoundError("lc0.exe not found, change lc0_path")

    try:
        process = subprocess.Popen(str(lc0_path),
                                   stdin=subprocess.PIPE,
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE,
                                   universal_newlines=True,
                                   bufsize=1,
                                   cwd=str(lc0_path.parent))

        process.stdin.write("uci\n")
        process.stdin.flush()

        while process.stdout.readline().strip() != "uciok":
            pass

        position_command = f"position fen {fen}" if fen else "position startpos"
        process.stdin.write(position_command + "\n")
        process.stdin.write(f"go movetime {str(time_limit)}\n")
        process.stdin.flush()

        best_move, score = None, None
        for line in process.stdout:
            line = line.strip()
            if line.startswith("info") and "score cp" in line:
                try:
                    score = int(line.split()[line.split().index("score") + 2])
                except (ValueError, IndexError):
                    pass
            elif line.startswith("bestmove"):
                best_move = line.split()[1]
                break

        return best_move, score

    except Exception as e:
        print(f"Error with the engine: {e}")
        return None, None

    finally:
        if "process" in locals():
            process.stdin.write("quit\n")
            process.stdin.flush()
            process.terminate()
            process.wait()

#### Function to Make a Move on the Board

In [ ]:
def make_move(move_str):
    try:
        move = chess.Move.from_uci(move_str)
        if move in board.legal_moves:
            board.push(move)
            return True
        else:
            print("Illegal move.")
            return False
    except ValueError:
        print("Invalid move format.")
        return False

#### Function to Update the Board and Show Information

In [ ]:
def show_position():
    with board_output:
        clear_output(wait=True)
        show_board(board)

    with info_output:
        clear_output(wait=True)
        if not board.is_game_over():
            best_move, best_score = get_best_move(board.fen())
            if best_move:
                best_move_uci = best_move
                all_legal_moves = list(board.legal_moves)
                random_moves = random.sample(all_legal_moves, 2)

                moves_with_best = [best_move_uci] + [move.uci() for move in random_moves]
                chosen_move = random.choice(moves_with_best)

                print(f"Recommended move: {best_move}")
                print(f"Position evaluation: {best_score/100 if best_score else 'undefined'}")
                print(f"{'White' if board.turn else 'Black'} is making a move")

            if board.move_stack:
                last_move = board.move_stack[-1]
                piece = board.piece_at(chess.parse_square(last_move.uci()[2:]))

                if piece:
                    piece_name = {
                        chess.PAWN: "pawn",
                        chess.KNIGHT: "knight",
                        chess.BISHOP: "bishop",
                        chess.KING: "king",
                        chess.QUEEN: "queen",
                        chess.ROOK: "rook"
                    }[piece.piece_type]
                    opponent_move_display.value = f"{piece_name} {last_move.uci()}"
        else:
            result = "Checkmate!" if board.is_checkmate() else "Stalemate!" if board.is_stalemate() else "Draw!"
            print(f"Game Over. {result}")

#### Function to Update the Board and Information

In [ ]:
def update_board_and_info():
    show_position()

#### Function for the Computer to Make a Move (for Black's Turn)

In [ ]:
def play_computer_move():
    if not board.is_game_over() and board.turn == chess.BLACK:
        best_move, best_score = get_best_move(board.fen())
        if best_move:
            best_move_uci = best_move
            all_legal_moves = list(board.legal_moves)
            random_moves = random.sample(all_legal_moves, 2)

            moves_with_best = [best_move_uci] + [move.uci() for move in random_moves]
            move = random.choice(moves_with_best)
            make_move(move)
            print(f"Computer made the move: {move}")
            update_board_and_info()

### Handler for Button Click

In [ ]:
def handle_button_click(action):
    if action == 'move':
        move_str = move_input.value.strip().lower()
        if move_str:
            if make_move(move_str):
                move_input.value = ''
            update_board_and_info()
            play_computer_move()

    elif action == 'undo':
        if len(board.move_stack) >= 2:
            board.pop()
            board.pop()
            opponent_move_display.value = ''
            update_board_and_info()

        else:
            with info_output:
                clear_output(wait=True)
                print("Cannot undo move.")

    elif action == 'new_game':
        board.reset()
        opponent_move_display.value = ''
        update_board_and_info()

### Button Handlers for Move, Undo, and New Game

In [ ]:
def on_move_button_click(b):
    handle_button_click("move")

def on_undo_button_click(b):
    handle_button_click("undo")

def on_new_game_button_click(b):
    handle_button_click("new_game")

### Create Input and Control Widgets

In [ ]:
move_input = widgets.Text(
    description='Move:',
    placeholder='e.g., e2e4',
    layout=widgets.Layout(width='200px')
)

move_button = widgets.Button(description="Make Move", button_style="success")
undo_button = widgets.Button(description="Undo Move", button_style="warning")
new_game_button = widgets.Button(description="New Game", button_style="info")


### Button Creation and Custom Layout Styling

In [ ]:
move_button = widgets.Button(
    description="Make Move",
    layout=widgets.Layout(width="200px", height="50px"),
    style={'button_color': '#9fc3d1', 'font_weight': 'bold'}
)

undo_button = widgets.Button(
    description="Undo Move",
    layout=widgets.Layout(width="200px", height="50px"),
    style={'button_color': '#4a656d', 'font_weight': 'bold'}
)

new_game_button = widgets.Button(
    description="New Game",
    layout=widgets.Layout(width="200px", height="50px"),
    style={'button_color': '#86abb6', 'font_weight': 'bold'}
)

#### Bind Buttons to Functions

In [ ]:
move_button.on_click(on_move_button_click)
undo_button.on_click(on_undo_button_click)
new_game_button.on_click(on_new_game_button_click)

### Layout for Controls

In [ ]:
controls = widgets.VBox([
    widgets.HBox([move_input, move_button]),
    widgets.HBox([undo_button, new_game_button]),
    opponent_move_display
])

### Display the Interface

In [ ]:
display(controls)
display(board_output)
display(info_output)

update_board_and_info()

## Visualization

### Automatic game from which I got data

In [ ]:
def play_full_game():
    board.reset()
    evaluations_white = []
    evaluations_black = []
    move_number = 1

    while not board.is_game_over():
        best_move, best_score = get_best_move(board.fen())

        if best_move:
            make_move(best_move)

            if board.turn == chess.WHITE:
                evaluations_white.append((best_score, move_number))
            else:
                evaluations_black.append((best_score, move_number))

            move_number += 1
            update_board_and_info()

            time.sleep(0.1)

    print("Game Over. Evaluations are stored.")
    return evaluations_white, evaluations_black

evaluations_white, evaluations_black = play_full_game()

### Position evaluation per move histogram
##### clearly shows how the position changed and which player had the advantage throughout the game. Each bar represents the position score after each move, where one group of bars represents the scores for White players and the other group represents the scores for Black players

In [ ]:
white_evaluations = evaluations_white
black_evaluations = evaluations_black

len_white = len(white_evaluations)
len_black = len(black_evaluations)

if len_white > len_black:
    black_evaluations.extend([(0, i) for i in range(len_black, len_white)])
elif len_black > len_white:
    white_evaluations.extend([(0, i) for i in range(len_white, len_black)])

white_evaluations = [score for score, _ in white_evaluations]
black_evaluations = [score for score, _ in black_evaluations]

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(white_evaluations))
width = 0.35

ax.bar(x - width/2, white_evaluations, width, label='White', color='#9fc3d1', align='center')
ax.bar(x + width/2, black_evaluations, width, label='Black', color='#4a656d', align='center')

ax.set_xlabel('Move Number')
ax.set_ylabel('Evaluation')
ax.set_title('Chess Game Evaluation for White and Black')

tick_interval = 5
ax.set_xticks(x[::tick_interval])
ax.set_xticklabels([f'{i+1}' for i in x[::tick_interval]])

plt.xticks(rotation=45)

ax.legend()

plt.tight_layout()
plt.show()